## DATA WRANGLING

In [1]:
# Import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pylab import rcParams

%matplotlib inline
import datetime as dt
import calendar
from datetime import datetime
rcParams["figure.figsize"] = 15, 6
import warnings
warnings.filterwarnings("ignore")
# import the sqlite package where we will connect and store the datasets
from sqlalchemy import create_engine
import sqlite3

In [2]:
# Create sqlalchemy engine and empty sprocket database
engine = create_engine("sqlite:///sprocket.db")

In [3]:
# Locate the excel filepath and read the sheet names
filepath = "C:/Users/USER/Downloads/KPMG_VI_New_raw_data_update_final.xlsx"
xls = pd.ExcelFile(filepath)
print(xls.sheet_names)

['Title Sheet', 'Transactions', 'NewCustomerList', 'CustomerDemographic', 'CustomerAddress']


In [4]:
# Iterate through the sheet names and read the datasets into a dataframe and save to sqlite3 database
for sheet_name in xls.sheet_names:
    sql_table = "tbl"+sheet_name
    df = pd.read_excel(filepath, sheet_name = sheet_name, index_col = None)
    # Connect to the database and store the excel datasets in the database
    conn = sqlite3.connect("sprocket.db")
    df.to_sql(sql_table, conn, if_exists = "replace", index = False)
    conn.close()

In [5]:
# Fetch all the tables in the database
conn = sqlite3.connect("sprocket.db")
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables = cur.fetchall()
print(tables)
conn.close()

[('tblTitle Sheet',), ('tblTransactions',), ('tblNewCustomerList',), ('tblCustomerDemographic',), ('tblCustomerAddress',)]


In [6]:
# Iterate through the tables in the database and extract the 
# Transactions, Customer Demographic, and Customer Address tables
tables = ['tblTransactions', 'tblCustomerDemographic', 'tblCustomerAddress']
dataset = {}
conn = sqlite3.connect("sprocket.db")

for table in tables:    
    query = f'SELECT * FROM {table}'
    dataset[table] = pd.read_sql_query(query, conn)

conn.close()

In [7]:
# Assign the tables to dataframes
dfTrans = dataset['tblTransactions']
dfDemo = dataset['tblCustomerDemographic']
dfAddr = dataset['tblCustomerAddress']

In [8]:
# Display first 2 entries of the Transaction dataframe
dfTrans.head(2)

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25 00:00:00,0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245


In [9]:
# Display first 2 entries of the Customer Demographic dataframe
dfDemo.head(2)

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11


In [10]:
# Display first 2 entries of the Customer Address dataframe
dfAddr.head(2)

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,customer_id,address,postcode,state,country,property_valuation
1,1,060 Morning Avenue,2016,New South Wales,Australia,10


In [11]:
# Loop through the dataframes and rename the first observation, columns
dataframes = [dfTrans, dfDemo, dfAddr]
for dataframe in dataframes:
    dataframe.columns = dataframe.iloc[0]
    # Drop the duplicate column names on index 0
    dataframe.drop(index = 0, inplace = True)
    dataframe.reset_index(drop = True, inplace = True)

In [12]:
# Display first 5 entries of the Transaction dataset
dfTrans.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25 00:00:00,0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
1,2,3,3120,2017-05-21 00:00:00,1,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
2,3,37,402,2017-10-16 00:00:00,0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
3,4,88,3135,2017-08-31 00:00:00,0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145
4,5,78,787,2017-10-01 00:00:00,1,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226


In [13]:
# Display the dimension of the Transaction dataset
dfTrans.shape

(20000, 13)

In [14]:
# Display first 5 entries of the Demographic dataset
dfDemo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
1,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
2,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
3,4,Talbot,None,Male,33,1961-10-03 00:00:00,None,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,None,Affluent Customer,N,NIL,Yes,8


In [15]:
# Display the dimension of the Demographic dataset
dfDemo.shape

(4000, 13)

In [16]:
# Display first 5 entries of the Address dataset
dfAddr.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [17]:
# Display the dimension of the Address dataset
dfAddr.shape

(3999, 6)

In [18]:
# View the information details about Transaction
dfTrans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   transaction_id           20000 non-null  object
 1   product_id               20000 non-null  object
 2   customer_id              20000 non-null  object
 3   transaction_date         20000 non-null  object
 4   online_order             19640 non-null  object
 5   order_status             20000 non-null  object
 6   brand                    19803 non-null  object
 7   product_line             19803 non-null  object
 8   product_class            19803 non-null  object
 9   product_size             19803 non-null  object
 10  list_price               20000 non-null  object
 11  standard_cost            19803 non-null  object
 12  product_first_sold_date  19803 non-null  object
dtypes: object(13)
memory usage: 1015.7+ KB


In [19]:
# Convert the transaction_date, list_price, and standard_cost to their correct datatypes
dfTrans["transaction_date"] = pd.to_datetime(dfTrans["transaction_date"])
prices = ["list_price", "standard_cost"]
for price in prices:
    dfTrans[price] = dfTrans[price].astype("float")

In [20]:
# Check for duplicated values
dfTrans.duplicated().sum()

0

In [21]:
# Display Transactions features with missing values and %
missingTransum = dfTrans.isnull().sum()
missingTranspercent = round(missingTransum/len(dfTrans) * 100, 1)
missingTrans = pd.concat([missingTransum, missingTranspercent], axis = 1)
missingTrans.columns = ["missing_counts", "missing_percent"]
missingTrans

,missing_counts,missing_percent
0,,
transaction_id,0,0.0
product_id,0,0.0
customer_id,0,0.0
transaction_date,0,0.0
online_order,360,1.8
order_status,0,0.0
brand,197,1.0
product_line,197,1.0
product_class,197,1.0


In [22]:
# Fill the online_order missing values with the mode of the feature
dfTrans["online_order"] = dfTrans["online_order"].fillna(dfTrans["online_order"].mode()[0])

In [23]:
# Delete rows with null values
dfTrans.dropna(axis = 0, inplace = True)

In [24]:
dfTrans.nunique()

0
transaction_id             19803
product_id                   101
customer_id                 3494
transaction_date             364
online_order                   2
order_status                   2
brand                          6
product_line                   4
product_class                  3
product_size                   3
list_price                   100
standard_cost                103
product_first_sold_date      100
dtype: int64

In [25]:
# Define a function for listing categorical variables so we check for consistency
def category(df):
    cat_list = []
    for x in df.columns:
        if df[x].dtypes == "object":
            cat_list.append(x)
    return cat_list

In [26]:
# Print unique values for categorical variables
cat_list = category(dfTrans)
for col in cat_list:
    print(col)
    print(dfTrans[col].unique())
    print("")
    print("_"*100)
    print("")

transaction_id
['1' '2' '3' ... '19998' '19999' '20000']

____________________________________________________________________________________________________

product_id
['2' '3' '37' '88' '78' '25' '22' '15' '67' '12' '5' '61' '35' '16' '79'
 '33' '54' '27' '82' '89' '64' '19' '72' '91' '1' '99' '0' '92' '14' '44'
 '76' '46' '55' '66' '81' '86' '32' '77' '96' '6' '47' '94' '93' '60' '28'
 '4' '38' '56' '58' '50' '80' '87' '84' '21' '31' '62' '17' '73' '45' '49'
 '95' '18' '70' '26' '39' '36' '98' '75' '42' '20' '24' '53' '65' '29'
 '11' '10' '7' '41' '9' '69' '90' '97' '100' '74' '71' '34' '57' '23' '51'
 '59' '63' '40' '8' '13' '30' '48' '68' '83' '43' '52' '85']

____________________________________________________________________________________________________

customer_id
['2950' '3120' '402' ... '2764' '3168' '3464']

____________________________________________________________________________________________________

online_order
['0' '1']

____________________________________

In [27]:
# Create a function to convert numbers into datetime format
def convert_datetime(num):
    dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + num - 2)
    return dt

In [28]:
# Convert product_first_sold_date to datetime
dfTrans["product_first_sold_date"] = dfTrans["product_first_sold_date"].astype("int")
dfTrans["product_first_sold_date"] = dfTrans["product_first_sold_date"].apply(convert_datetime)
dfTrans.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,1,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [29]:
# Engineer new features for the dataset
# Create the profit feature
dfTrans["profit"] = dfTrans["list_price"] - dfTrans["standard_cost"]
# Create Transaction year
dfTrans["transaction_year"] = dfTrans["transaction_date"].dt.year
# Create Transaction month
dfTrans["transaction_month"] = dfTrans["transaction_date"].dt.month_name()
# Create Transaction day
dfTrans["transaction_day"] = dfTrans["transaction_date"].dt.day
# Create Transaction day name
dfTrans["day_of_the_week"] = dfTrans["transaction_date"].dt.day_name()

In [30]:
dfTrans.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,profit,transaction_year,transaction_month,transaction_day,day_of_the_week
0,1,2,2950,2017-02-25,0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02,17.87,2017,February,25,Saturday
1,2,3,3120,2017-05-21,1,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03,1702.55,2017,May,21,Sunday
2,3,37,402,2017-10-16,0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20,1544.61,2017,October,16,Monday
3,4,88,3135,2017-08-31,0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16,817.36,2017,August,31,Thursday
4,5,78,787,2017-10-01,1,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10,1055.82,2017,October,1,Sunday


In [31]:
# Convert transaction_day to object type
dfTrans["transaction_day"] = dfTrans["transaction_day"].astype("str")

In [32]:
# View the information details about Transaction
dfTrans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19803 entries, 0 to 19999
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           19803 non-null  object        
 1   product_id               19803 non-null  object        
 2   customer_id              19803 non-null  object        
 3   transaction_date         19803 non-null  datetime64[ns]
 4   online_order             19803 non-null  object        
 5   order_status             19803 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               19803 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [33]:
# View the information details about Demographic
dfDemo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   customer_id                          4000 non-null   object
 1   first_name                           4000 non-null   object
 2   last_name                            3875 non-null   object
 3   gender                               4000 non-null   object
 4   past_3_years_bike_related_purchases  4000 non-null   object
 5   DOB                                  3913 non-null   object
 6   job_title                            3494 non-null   object
 7   job_industry_category                3344 non-null   object
 8   wealth_segment                       4000 non-null   object
 9   deceased_indicator                   4000 non-null   object
 10  default                              3698 non-null   object
 11  owns_car                             4000 n

In [34]:
# Convert the DOB, wealth_segment, and past_3_years_bike_related_purchases to their correct datatypes
dfDemo["DOB"] = pd.to_datetime(dfDemo["DOB"])
wealth = ['High Net Worth', 'Affluent Customer', 'Mass Customer']
class_wealth = pd.api.types.CategoricalDtype(ordered = True, categories = wealth)
dfDemo["wealth_segment"] = dfDemo["wealth_segment"].astype(class_wealth)
dfDemo["past_3_years_bike_related_purchases"] = dfDemo["past_3_years_bike_related_purchases"].astype("int")

In [35]:
# Check for inaccuracy in the DOB column
dfDemo.sort_values(by = "DOB", ascending = True).head(5)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
33,34,Jephthah,Bachmann,U,59,1843-12-21,Legal Assistant,IT,Affluent Customer,N,None,No,20
719,720,Darrel,Canet,Male,67,1931-10-23,Recruiting Manager,Retail,Affluent Customer,N,1;DROP TABLE users,No,6
1091,1092,Katlin,Creddon,Female,56,1935-08-22,VP Quality Control,Retail,Mass Customer,N,ì¬íê³¼íì ì´íì°êµ¬ì,No,5
3409,3410,Merrili,Brittin,Female,93,1940-09-22,None,Property,Mass Customer,N,á,No,16
2412,2413,Abbey,Murrow,Male,27,1943-08-11,Environmental Specialist,Manufacturing,High Net Worth,N,á,Yes,17


In [36]:
# Check for duplicated values
dfDemo.duplicated().sum()

0

In [37]:
# Display Demographic features with missing values and %
missingDemosum = dfDemo.isnull().sum()
missingDemopercent = round(missingDemosum/len(dfDemo) * 100, 1)
missingDemo = pd.concat([missingDemosum, missingDemopercent], axis = 1)
missingDemo.columns = ["missing_counts", "missing_percent"]
missingDemo

,missing_counts,missing_percent
0,,
customer_id,0,0.0
first_name,0,0.0
last_name,125,3.1
gender,0,0.0
past_3_years_bike_related_purchases,0,0.0
DOB,87,2.2
job_title,506,12.6
job_industry_category,656,16.4
wealth_segment,0,0.0


In [38]:
# Drop the irrelevant 'default' column
dfDemo.drop("default", axis = 1, inplace = True)

In [39]:
# Subset the DOB and tenure feature and see how distributed the isnull value is
dfDemo[dfDemo["DOB"].isnull()][["DOB", "tenure"]].isnull().sum()

0
DOB       87
tenure    87
dtype: int64

In [40]:
# Fill the job_industry_category, job_title, and last_name missing values with the mode of the feature
lists = ["job_industry_category", "job_title", "last_name"]
for mod in lists:
    dfDemo[mod] = dfDemo[mod].fillna(dfDemo[mod].mode()[0])

In [41]:
# Drop the other rows with nan values
dfDemo.dropna(axis = 0, inplace = True)

In [42]:
dfDemo.nunique()

0
customer_id                            3913
first_name                             3079
last_name                              3647
gender                                    6
past_3_years_bike_related_purchases     100
DOB                                    3448
job_title                               195
job_industry_category                     9
wealth_segment                            3
deceased_indicator                        2
owns_car                                  2
tenure                                   22
dtype: int64

In [43]:
# Print unique values for categorical variables
cat_list = category(dfDemo)
for col in cat_list:
    print(col)
    print(dfDemo[col].unique())
    print("")
    print("_"*100)
    print("")

customer_id
['1' '2' '3' ... '3997' '3999' '4000']

____________________________________________________________________________________________________

first_name
['Laraine' 'Eli' 'Arlin' ... 'Rolph' 'Stephie' 'Rusty']

____________________________________________________________________________________________________

last_name
['Medendorp' 'Bockman' 'Dearle' ... 'Iapico' 'Halgarth' 'Oldland']

____________________________________________________________________________________________________

gender
['F' 'Male' 'Female' 'U' 'Femal' 'M']

____________________________________________________________________________________________________

job_title
['Executive Secretary' 'Administrative Officer' 'Recruiting Manager'
 'Business Systems Development Analyst' 'Senior Editor' 'Media Manager I'
 'Senior Quality Engineer' 'Nuclear Power Engineer' 'Developer I'
 'Account Executive' 'Junior Executive' 'Media Manager IV'
 'Sales Associate' 'Professor' 'Geological Engineer' 'Project Manager'

In [44]:
# Check for inaccuracy in the DOB column
dfDemo.sort_values(by = "DOB").head(2)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
33,34,Jephthah,Bachmann,U,59,1843-12-21,Legal Assistant,IT,Affluent Customer,N,No,20
719,720,Darrel,Canet,Male,67,1931-10-23,Recruiting Manager,Retail,Affluent Customer,N,No,6


In [45]:
# Drop the DOB "1843" row
dfDemo.drop(index = [33], axis = 0, inplace = True)

In [46]:
# Correct the inconsistencies in the gender column
dfDemo["gender"] = dfDemo["gender"].replace("F", "Female")
dfDemo["gender"] = dfDemo["gender"].replace("Femal", "Female")
dfDemo["gender"] = dfDemo["gender"].replace("M", "Male")

In [47]:
# Correct inaccuracy in the spelling of Agriculture
dfDemo["job_industry_category"] = dfDemo["job_industry_category"].replace("Argiculture", "Agriculture")

In [48]:
# Convert tenure to int datatype
dfDemo["tenure"] = dfDemo["tenure"].astype("int")

In [49]:
# Convert DOB to age using a function
def to_age(birth):
    today = dt.date.today()
    age = today.year - birth.year - ((today.month, today.day) < (birth.month, birth.day))
    return age

In [50]:
# Applying the function to the DOB column
dfDemo["age"] = dfDemo["DOB"].apply(lambda x: to_age(x))

In [51]:
dfDemo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11,70
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16,42
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15,69
3,4,Talbot,Pristnor,Male,33,1961-10-03,Business Systems Development Analyst,IT,Mass Customer,N,No,7,62
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Manufacturing,Affluent Customer,N,Yes,8,46


In [52]:
# Drop the DOB column
dfDemo.drop("DOB", axis = 1, inplace = True)

In [53]:
# View information about the dataset
dfDemo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3912 entries, 0 to 3999
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   customer_id                          3912 non-null   object  
 1   first_name                           3912 non-null   object  
 2   last_name                            3912 non-null   object  
 3   gender                               3912 non-null   object  
 4   past_3_years_bike_related_purchases  3912 non-null   int32   
 5   job_title                            3912 non-null   object  
 6   job_industry_category                3912 non-null   object  
 7   wealth_segment                       3912 non-null   category
 8   deceased_indicator                   3912 non-null   object  
 9   owns_car                             3912 non-null   object  
 10  tenure                               3912 non-null   int32   
 11  age              

In [54]:
# View the information details about Address dataset
dfAddr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   object
 1   address             3999 non-null   object
 2   postcode            3999 non-null   object
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   object
dtypes: object(6)
memory usage: 93.8+ KB


In [55]:
# Check for duplicated values
dfAddr.duplicated().sum()

0

In [56]:
# Display Address features with missing values
missingAddr = dfAddr.isnull()

for column in missingAddr.columns.values.tolist():
    print(column)
    print(missingAddr[column].value_counts())
    print("_"*100)
    print("")

customer_id
False    3999
Name: customer_id, dtype: int64
____________________________________________________________________________________________________

address
False    3999
Name: address, dtype: int64
____________________________________________________________________________________________________

postcode
False    3999
Name: postcode, dtype: int64
____________________________________________________________________________________________________

state
False    3999
Name: state, dtype: int64
____________________________________________________________________________________________________

country
False    3999
Name: country, dtype: int64
____________________________________________________________________________________________________

property_valuation
False    3999
Name: property_valuation, dtype: int64
____________________________________________________________________________________________________



In [57]:
# Print unique values for categorical variables
cat_list = category(dfAddr)
for col in cat_list:
    print(col)
    print(dfAddr[col].unique())
    print("")
    print("_"*100)
    print("")

customer_id
['1' '2' '4' ... '4001' '4002' '4003']

____________________________________________________________________________________________________

address
['060 Morning Avenue' '6 Meadow Vale Court' '0 Holy Cross Court' ...
 '87 Crescent Oaks Alley' '8194 Lien Street' '320 Acker Drive']

____________________________________________________________________________________________________

postcode
['2016' '2153' '4211' '2448' '3216' '2210' '2650' '2023' '3044' '4557'
 '3799' '2760' '2428' '3331' '3058' '2135' '2233' '2444' '4413' '4740'
 '3218' '4868' '4116' '2519' '2756' '2170' '4005' '4127' '2535' '4726'
 '4805' '2212' '2207' '3350' '3931' '4018' '4670' '2480' '3212' '2322'
 '3198' '2047' '4503' '2145' '4350' '2142' '2029' '2127' '4280' '2166'
 '2046' '2026' '2765' '2749' '2745' '3752' '2217' '2230' '3046' '3183'
 '2070' '2213' '4208' '2774' '4133' '4213' '4170' '2073' '3561' '2330'
 '3977' '2204' '2086' '2226' '2075' '2038' '2069' '2100' '3133' '3070'
 '2777' '2125' '2766' '22

In [58]:
# Correct the inconsistencies in labeled data values
dfAddr["state"] = dfAddr["state"].replace('New South Wales', "NSW")
dfAddr["state"] = dfAddr["state"].replace('Victoria', "VIC")
# Test
dfAddr["state"].unique()

array(['NSW', 'QLD', 'VIC'], dtype=object)

In [59]:
# Convert property_valuation to int datatype
dfAddr["property_valuation"] = dfAddr["property_valuation"].astype("int")

In [60]:
# Drop the country column since we are dealing with Australia dataset
dfAddr.drop("country", axis = 1, inplace = True)

In [61]:
# Primary key for joining the datasets
all_columns = pd.Series(list(dfTrans) + list(dfDemo) + list(dfAddr))
all_columns[all_columns.duplicated()].unique()

array(['customer_id'], dtype=object)

In [62]:
# Merge Transactions, Demographic, and Address dataframes together using the merge function
dfTransaction = pd.merge(dfTrans, dfDemo, on = "customer_id", how = "inner")
dfCustomer = pd.merge(dfTransaction, dfAddr, on = "customer_id", how = "inner")

In [63]:
# View information on the joined dataset
dfCustomer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19327 entries, 0 to 19326
Data columns (total 33 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   transaction_id                       19327 non-null  object        
 1   product_id                           19327 non-null  object        
 2   customer_id                          19327 non-null  object        
 3   transaction_date                     19327 non-null  datetime64[ns]
 4   online_order                         19327 non-null  object        
 5   order_status                         19327 non-null  object        
 6   brand                                19327 non-null  object        
 7   product_line                         19327 non-null  object        
 8   product_class                        19327 non-null  object        
 9   product_size                         19327 non-null  object        
 10  list_price

In [64]:
dfCustomer.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,...,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age,address,postcode,state,property_valuation
0,1,2,2950,2017-02-25,0,Approved,Solex,Standard,medium,medium,...,Financial Services,Mass Customer,N,Yes,10,68,984 Hoepker Court,3064,VIC,6
1,11065,1,2950,2017-10-16,0,Approved,Giant Bicycles,Standard,medium,medium,...,Financial Services,Mass Customer,N,Yes,10,68,984 Hoepker Court,3064,VIC,6
2,18923,62,2950,2017-04-26,0,Approved,Solex,Standard,medium,medium,...,Financial Services,Mass Customer,N,Yes,10,68,984 Hoepker Court,3064,VIC,6
3,2,3,3120,2017-05-21,1,Approved,Trek Bicycles,Standard,medium,large,...,Health,Mass Customer,N,Yes,10,44,4 Shopko Circle,2196,NSW,5
4,6862,4,3120,2017-10-05,0,Approved,Giant Bicycles,Standard,high,medium,...,Health,Mass Customer,N,Yes,10,44,4 Shopko Circle,2196,NSW,5


In [65]:
# Save dataset to csv file
path = "C:/Users/USER/Documents/Udacity/Sprocket/Exploratory Analysis/Customer_Wrangled.csv"
dfCustomer.to_csv(path, index = False)